# ASSIGNMENT 5: Frequency sampling, FIR filtering
In the frequency sampling method samples from the desired frequency response with intervals of $f_s/N$ are
taken. The impulse response of the filter is formed by inverse DFT from these samples. The continuous
frequency response of the filter goes along the defined points, but between them it gets undesired values. If
some frequency samples are also defined in the transition band, the Gibb’s phenomenon can be reduced.

Start by running the code block below to load the Python packages required to complete this assignment. To run the code, select the code block and push Ctrl+Enter or click the Run button.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
%matplotlib inline

**Task 5.1** Generate ideal frequency response sample vectors H0_ideal and H2_ideal of a low-pass filter
using function *frqid*. For example:
```python
H0_ideal = frqid(fp, L, T)
```
The function returns an array of samples in the frequency domain. Use the following parameters: passband edge frequency fp = 0.225, length of the filter L = 29
and number of transition band samples T = 0 for H0_ideal and T = 2 for H2_ideal.

In [3]:
def frqid(fp, L, T):
    """
    frqid(fp, L, T)
    
    fp: normalized passband edge frequency [0, 0.5], fs = 1
    L: length of signal in samples
    T: number of transition band samples 0 - 3
    
    returns:
    H: sampled ideal frequency response with T transition band samples
    """
    # sampled amplitude response of the optimal filter
    passb = (np.fix(fp*L) + 1).astype('int')
    if (L % 2) == 0:
        s = -1
    else:
        s = 1
    
    # transition band samples 0, 1 and 3
    if (T == 0):
        Ad = np.concatenate((np.ones(passb), np.zeros(L - 2*passb + 1),
                             s*np.ones(passb-1)), axis=None) 
    elif (T == 1):
        Ad = np.concatenate((np.ones(passb-1), 0.5,
                             np.zeros(L - 2*passb + 1), 0.5,
                             s*np.ones(passb-2)), axis=None)
    elif (T == 2):
        Ad = np.concatenate((np.ones(passb-1), 0.67, 0.33,
                             np.zeros(L - 2*passb - 1), 0.33, 0.67,
                             s*np.ones(passb-2)), axis=None)
    elif (T == 3):
        Ad = np.concatenate((p.ones(passb-1), 0.75, 0.5, 0.25,
                             np.zeros(L - 2*passb - 3), 0.25, 0.5, 0.75,
                             s*np.ones(passb-2)), axis=None)
    else:
        print('Error: Number of transition band samples 0 - 3.')
    
    # sampled frequency response of the optimal filter
    M = (L-1) / 2
    k = np.arange(0, L)
    p = np.exp(2 * np.pi * 1j * (-M) * k/L)
    return Ad * p

In [4]:
# form the ideal sampled frequency responses
fp0 = 0.225
L0 = 29
T0 = 0
H0_ideal = frqid(fp0, L0, T0)

fp2 = 0.225
L2 = 29
T2 = 2
H2_ideal = frqid(fp2, L2, T2)


Calculate the impulse responses h0 and h2 from the samples of the ideal frequency responses using the
inverse Fourier transform (use the function `np.fft.ifft`). Remove the negligible imaginary parts (that should be zero) using
commands
```python
h0 = np.real(h0)
h2 = np.real(h2)
```


In [ ]:
# calculcate the inverse dft for the sampled frequency responses


# take the real part of each resulting impulse response



Run the following commands to plot the final 512-point amplitude responses of the filters in the same figures with the original samples from the ideal frequency response:

In [ ]:
H0 = np.fft.fft(h0, 512)
H2 = np.fft.fft(h2, 512)
ff = np.arange(0, len(H0), dtype=float) / len(H0)
ff_ideal = np.arange(0, len(H0_ideal), dtype=float) / len(H0_ideal)

fig, ax = plt.subplots(2, 1)
ax[0].plot(ff[0:256], abs(H0[0:256]))
ax[0].stem(ff_ideal[0:15], abs(H0_ideal[0:15]))
ax[0].set_title("H0 and H0_ideal")
ax[1].plot(ff[0:256], abs(H2[0:256]))
ax[1].stem(ff_ideal[0:15], abs(H2_ideal[0:15]))
ax[1].set_title("H2 and H2_ideal")
fig.tight_layout()

If your design is correct, the final amplitude response curves will go through the points of the ideal amplitude
response.

**What is the advantage and disadvantage of also taking samples in the transition band?**

`* Type your answer here *`

**Task 5.2** Load a set of ECG data with the following commands:

In [ ]:
# load the signals ECG_clean and ECG_noisy
ECG_clean = np.loadtxt("ECG_clean.dat", delimiter=',')
ECG_noisy = np.loadtxt("ECG_noisy.dat", delimiter=',')

You have two signals: ECG_clean and ECG_noisy. The latter contains noise in addition to information. Your task is to filter the high
frequency noise away from the disturbed signal using your low-pass filter with impulse response h2
from the previous task.
The filtering can be done by using convolution. For example, the command
```python
y = signal.convolve(x, h)
```
calculates the convolution between the signal x and h, the impulse response of a filter, and stores the resulting signal in y.
Plot the amplitude spectra (= the absolute value of a DFT) of the following three signals to one
figure using the *subplots* or *subplot* command.

- the original noisy ecg signal (ECG_noisy)
- the clean ecg signal (ECG_clean)
- the filtered signal

Use the function *plot* to draw the amplitude spectra. Plotting them
with *stem* doesn’t look good.

In [ ]:
# filter ECG_noisy using the low pass filter


# plot ECG_noisy, ECG_clean and the filtered signal



### Instructions for submission
1. Make sure that your code produces the desired results from scratch by selecting Kernel -> Restart & Run all. This will clear the output of your code and run all the code cells again from top to bottom.

2. Rename this notebook to `DF_Assignment5_firstname_lastname_studentnumber.ipynb`.

3. Submit only the resulting notebook (.ipynb file) to the return box in Moodle. Send your submission before the deadline to receive additional week exam points.